In [13]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input
from keras.layers import LSTM

from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import h5py
import numpy as np
import random
import json
import sys

In [2]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

total chars: 57
nb sequences: 200285


In [4]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
X.shape, y.shape

((200285, 40, 57), (200285, 57))

In [5]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [7]:
model.fit(X, y,
          batch_size=256,
          epochs=40)


--------------------------------------------------
Iteration 1
Epoch 1/5
200285/200285 [==============================] - 110s 550us/step - loss: 2.0688 - acc: 0.3986
Epoch 2/5
200285/200285 [==============================] - 114s 571us/step - loss: 1.6560 - acc: 0.5067
Epoch 3/5
200285/200285 [==============================] - 124s 619us/step - loss: 1.5434 - acc: 0.5371
Epoch 4/5
200285/200285 [==============================] - 125s 625us/step - loss: 1.4853 - acc: 0.5519
Epoch 5/5
200285/200285 [==============================] - 126s 631us/step - loss: 1.4495 - acc: 0.5615

----- diversity: 0.2
----- Generating with seed: "").

258. corruption--as the indication "
").

258. corruption--as the indication of the subst and the standing to extent of the sense of the stated to the suffering of the subjection of the sense of the sense of the subjection of the soul of the sublimating in the states of the propress of an and the decided to the fact to the sense of the state of the standed t

/Users/douwe/Dropbox/Douwe/Proj/notebooks/venv3/lib/python3.5/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 speul
of reprection somebom awaze, because him as the logecs the
scor
alwidry. bas
has that paserise
stra
who
have a epension the

--------------------------------------------------
Iteration 4
Epoch 1/5
200285/200285 [==============================] - 102s 511us/step - loss: 1.3130 - acc: 0.5968
Epoch 2/5
200285/200285 [==============================] - 28837s 144ms/step - loss: 1.3095 - acc: 0.5988
Epoch 3/5
200285/200285 [==============================] - 107s 536us/step - loss: 1.3039 - acc: 0.5979
Epoch 4/5
200285/200285 [==============================] - 97s 482us/step - loss: 1.2992 - acc: 0.6004
Epoch 5/5
200285/200285 [==============================] - 100s 500us/step - loss: 1.2943 - acc: 0.6006

----- diversity: 0.2
----- Generating with seed: "be
returned.

183. "i am affected, not b"
be
returned.

183. "i am affected, not be the contrary and family, and all the contrary to the strange and sensations of the sense of the delight in the probably more proposite of the strange

ala general sequence as has to ame is for time wamech loud in spirate that which sie is for an 
estle ty to 'esultame; hi sally inever generally in masis. suppose the judg--well-stcou

--------------------------------------------------
Iteration 7
Epoch 1/5
200285/200285 [==============================] - 101s 505us/step - loss: 1.2558 - acc: 0.6109
Epoch 2/5
200285/200285 [==============================] - 101s 506us/step - loss: 1.2526 - acc: 0.6125
Epoch 3/5
200285/200285 [==============================] - 101s 506us/step - loss: 1.2508 - acc: 0.6141
Epoch 4/5
200285/200285 [==============================] - 103s 514us/step - loss: 1.2486 - acc: 0.6133
Epoch 5/5
200285/200285 [==============================] - 103s 515us/step - loss: 1.2455 - acc: 0.6140

----- diversity: 0.2
----- Generating with seed: "d the dangerous
desire to see him also i"
d the dangerous
desire to see him also in the sentince and the presented to the presented and the presented to the processions of self deli

In [17]:
model.save('keras_js/nietzsche.h5')
with open('keras_js/chars.js', 'w') as fout:
    fout.write('maxlen = ' + str(maxlen) + '\n')
    fout.write('num_chars = ' + str(len(chars)) + '\n')
    fout.write('char_indices = ' + json.dumps(char_indices, indent=2) + '\n')
    fout.write('indices_char = ' + json.dumps(indices_char, indent=2) + '\n')
